# 🎮 GPU-as-a-Service Orchestrator

**Platform Engineering Standard Operating Procedure (SOP)**

This notebook demonstrates **RHOAI 3.0 GPU quota management** using native OpenShift `oc` commands that any Platform Engineer can use.

---

## 📊 Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    Kueue Quota Management                       │
│                                                                 │
│   ClusterQueue: rhoai-main-queue                                │
│   ├── nominalQuota: 5 GPUs                                      │
│   └── ResourceFlavors: nvidia-l4-1gpu, nvidia-l4-4gpu           │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📺 Setup: Open Monitoring Dashboard

**Before running the demo**, open the RHOAI Dashboard in another tab:

1. Navigate to **RHOAI Dashboard** → **Distributed Workloads**
2. Select project: **private-ai**
3. Watch the **Workloads** table for status changes

This provides real-time visibility into Kueue admission decisions.


---

## 🔍 Step 0: Check Current State

Let's verify the baseline: **5/5 GPUs used** (BF16 + INT4)


In [ ]:
# View all InferenceServices and their replica counts
! oc get inferenceservice -n private-ai


In [ ]:
# Check Kueue workloads - shows admitted vs pending
! oc get workloads -n private-ai


In [ ]:
# Check deployments
! oc get deployments -n private-ai


In [ ]:
# Check running pods
! oc get pods -n private-ai -l serving.kserve.io/inferenceservice


In [ ]:
# Scale up Deployment - IMMEDIATE
! oc scale deployment {name}-predictor -n private-ai --replicas=1


In [ ]:
# Scale down Deployment - IMMEDIATE
! oc scale deployment {name}-predictor -n private-ai --replicas=0


---

## 🔄 Reset: Return to Baseline State

Return to the original "saturated" state for the next demo run.


In [ ]:
# Reset all models to baseline - IMMEDIATE via Deployments
! oc scale deployment mistral-3-bf16-predictor -n private-ai --replicas=1
! oc scale deployment mistral-3-int4-predictor -n private-ai --replicas=1
! oc scale deployment devstral-2-predictor -n private-ai --replicas=0
! oc scale deployment gpt-oss-20b-predictor -n private-ai --replicas=0
! oc scale deployment granite-8b-agent-predictor -n private-ai --replicas=0


In [ ]:
# Verify baseline restored
! oc get inferenceservice -n private-ai


---

## 📖 Technical Reference

### Why `oc scale deployment` (not `oc scale inferenceservice`)

RHOAI 3.0 uses **RawDeployment mode** by default (not Knative Serving). This means:
- No `ksvc` (Knative Services) are created
- Each InferenceService creates a Deployment named `{name}-predictor`
- `oc scale inferenceservice` doesn't work (scale subresource not implemented)

**The elegant solution:** Scale the underlying Deployment directly!

```bash
# Scale DOWN (immediate - pod terminates instantly)
oc scale deployment {name}-predictor -n private-ai --replicas=0

# Scale UP (Kueue manages admission via SchedulingGates)
oc scale deployment {name}-predictor -n private-ai --replicas=1
```

### Deployment Naming Convention

| InferenceService | Deployment |
|-----------------|------------|
| `mistral-3-bf16` | `mistral-3-bf16-predictor` |
| `mistral-3-int4` | `mistral-3-int4-predictor` |
| `devstral-2` | `devstral-2-predictor` |
| `granite-8b-agent` | `granite-8b-agent-predictor` |
| `gpt-oss-20b` | `gpt-oss-20b-predictor` |

### How Kueue Intercepts Scale Events

Because the `private-ai` namespace has `kueue.openshift.io/managed=true`:

1. Scale command adjusts Deployment replicas → triggers pod creation
2. Kueue intercepts via **SchedulingGates** (pods start as `SchedulingGated`)
3. Kueue checks against `ClusterQueue` quota (5 GPUs)
4. **If under quota:** Pod admitted immediately, starts running
5. **If over quota:** Pod held in `SchedulingGated` until resources free

### Documentation Links

- [RHOAI 3.0: Deploying Models](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/3.0/html-single/deploying_models/index)
- [RHOAI 3.0: Managing Distributed Workloads](https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/3.0/html-single/working_with_distributed_workloads/index)
- [Kueue Documentation](https://kueue.sigs.k8s.io/)
